In [ ]:
######################
# Jason Hemedinger
# Argonne National Laboratory
# This notebook will pull radar data from Amazon S3

In [1]:
import pyart
import boto3
import tempfile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from botocore.handlers import disable_signing
from tint import Cell_tracks, animate


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
def get_current_scan(station_date, key_index):
    '''
    Function will pull the latest radar scan from any radar site using 
    Amazon S3.
    ----------
    station_date= 'year/month/day/station'
    year = YYYY e.g. 1776
    month = mm e.g. 07
    day = dd e.g. 04
    site = xxxx e.g. kdix
    '''
    bucket = 'noaa-nexrad-level2'
    s3 = boto3.resource('s3')
    s3.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
    
    aws_radar = s3.Bucket(bucket)
    
    for obj in aws_radar.objects.filter(Prefix= station_date):
        '{0}:{1}'.format(aws_radar.name, obj.key)
    my_list_of_keys = [this_object.key for this_object in aws_radar.objects.filter(Prefix= station_date)]
    keys = my_list_of_keys[key_index:]
    print(my_list_of_keys[key_index:])
    
    
    return aws_radar, keys



In [3]:
aws_radar, keys = get_current_scan('2018/06/06/KLOT', key_index=-30)

['2018/06/06/KLOT/KLOT20180606_155419_V06', '2018/06/06/KLOT/KLOT20180606_155826_V06', '2018/06/06/KLOT/KLOT20180606_160232_V06', '2018/06/06/KLOT/KLOT20180606_160708_V06', '2018/06/06/KLOT/KLOT20180606_161143_V06', '2018/06/06/KLOT/KLOT20180606_161626_V06', '2018/06/06/KLOT/KLOT20180606_162057_V06', '2018/06/06/KLOT/KLOT20180606_162531_V06', '2018/06/06/KLOT/KLOT20180606_163020_V06', '2018/06/06/KLOT/KLOT20180606_163455_V06', '2018/06/06/KLOT/KLOT20180606_163929_V06', '2018/06/06/KLOT/KLOT20180606_164418_V06', '2018/06/06/KLOT/KLOT20180606_164907_V06', '2018/06/06/KLOT/KLOT20180606_165352_V06', '2018/06/06/KLOT/KLOT20180606_165835_V06', '2018/06/06/KLOT/KLOT20180606_170324_V06', '2018/06/06/KLOT/KLOT20180606_170804_V06', '2018/06/06/KLOT/KLOT20180606_171238_V06', '2018/06/06/KLOT/KLOT20180606_171712_V06', '2018/06/06/KLOT/KLOT20180606_172201_V06', '2018/06/06/KLOT/KLOT20180606_172651_V06', '2018/06/06/KLOT/KLOT20180606_173125_V06', '2018/06/06/KLOT/KLOT20180606_173604_V06', '2018/06/0

In [4]:
max_lon = -87
min_lon = -90
max_lat = 43
min_lat = 41
lat_lines = np.arange(min_lat, max_lat, .2)
lon_lines = np.arange(min_lon, max_lon, .5)

In [5]:
def animate(nframe):
    plt.clf()
    localfile = tempfile.NamedTemporaryFile()
    aws_radar.download_file(keys[nframe], localfile.name)
    # Only pulling two scans for the sake of time and memory.
    radar = pyart.io.read(localfile.name)
    display = pyart.graph.RadarMapDisplay(radar)
    # Delete radar after use to save memory.
    del radar
    display.plot_ppi_map('reflectivity', sweep=0, resolution='l',
                         vmin=-8, vmax=64, mask_outside=False,
                         cmap=pyart.graph.cm.NWSRef, min_lat=min_lat,
                         max_lat=max_lat, min_lon=min_lon, max_lon=max_lon,
                         lat_lines=lat_lines, lon_lines=lon_lines)
    display.basemap.drawcounties()
    display.plot_point(-87.981810, 41.713969 , label_text='ANL', color = 'k')
fig = plt.figure(figsize=(10, 8))
anim_klot = animation.FuncAnimation(fig, animate,
                                    frames=len(keys))
anim_klot.save('reflectivity_animation.gif',
               writer='imagemagick', fps=2)
plt.close()